In [7]:
# 1. 패키지 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
def load_data():
    승하차_파일 = "../../data/결과/승하차/통합/1호선_승하차인원_통합.csv"
    혼잡도_파일 = "../../data/결과/혼잡도/통합/1호선_혼잡도_통합.csv"
    승하차_df = pd.read_csv(승하차_파일, encoding="euc-kr")
    혼잡도_df = pd.read_csv(혼잡도_파일, encoding="euc-kr")
    # 시간컬럼명 0패딩 통일
    def fix_timecols(cols):
        return [c if ':' not in c else c.zfill(5) for c in cols]
    승하차_df.columns = fix_timecols(승하차_df.columns)
    혼잡도_df.columns = fix_timecols(혼잡도_df.columns)
    return 승하차_df, 혼잡도_df


In [9]:
# 3. 승하차 데이터를 상행/하행으로 2배 복제
def duplicate_for_directions(승하차):
    dfs = []
    for direction in ["상행", "하행"]:
        tmp = 승하차.copy()
        tmp['방향'] = direction
        dfs.append(tmp)
    return pd.concat(dfs, ignore_index=True)


In [10]:
# 전처리 함수
def standardize_columns(df, do_direction=True):
    df["평일주말"] = df["평일주말"].astype(str).str.strip()
    df["구분"] = df["구분"].astype(str).str.strip()
    df["역번호"] = df["역번호"].astype(str).str.strip()
    if do_direction and "방향" in df.columns:
        df["방향"] = df["방향"].astype(str).str.strip()
    return df




In [11]:
# 5. AI 딥러닝 학습 데이터 생성
def build_dl_dataset(승하차, 혼잡도, 정원=2000):
    rows = []
    시간컬럼들 = [col for col in 승하차.columns if ':' in col]
    fail, total = 0, 0
    for idx, row in 승하차.iterrows():
        for col in 시간컬럼들:
            total += 1
            hour = int(col.split(":")[0])
            평일주말 = 1 if row["평일주말"] == "주말" else 0
            상행 = 1 if row["방향"] == "상행" else 0
            승차 = 1 if row["구분"] == "승차" else 0
            matched = 혼잡도[
                (혼잡도["역번호"].astype(str).str.strip() == row["역번호"]) &
                (혼잡도["평일주말"].astype(str).str.strip() == row["평일주말"]) &
                (혼잡도["구분"].astype(str).str.strip() == row["방향"])
            ]
            if matched.empty or col not in 혼잡도.columns:
                fail += 1
                continue
            try:
                congestion = float(matched.iloc[0][col])
                승하차인원 = float(row[col])
            except Exception as e:
                fail += 1
                continue
            y = int(congestion * 정원 / 100)
            rows.append([
                int(row["역번호"]), hour, 평일주말, 상행, 승차, 승하차인원, congestion, y
            ])
    print(f"> 학습데이터 생성: {len(rows)}행, 실패 {fail} / 총시도 {total}")
    cols = ["역번호", "hour", "평일주말", "상행", "승차", "승하차인원", "혼잡도", "target"]
    return pd.DataFrame(rows, columns=cols)

In [12]:
# 6. 전체 파이프라인 (학습+예측)
승하차, 혼잡도 = load_data()
승하차_expanded = duplicate_for_directions(승하차)
승하차_expanded = standardize_columns(승하차_expanded)
혼잡도 = standardize_columns(혼잡도)
df = build_dl_dataset(승하차_expanded, 혼잡도)
print(df.head())
if df.shape[0]==0:
    raise ValueError("학습데이터 생성 실패! 컬럼 또는 값 일치 여부 확인 필요.")


> 학습데이터 생성: 2629600행, 실패 0 / 총시도 2629600
   역번호  hour  평일주말  상행  승차   승하차인원   혼잡도  target
0  150     5     1   1   1   441.0   2.3      46
1  150     6     1   1   1   399.0  10.2     204
2  150     7     1   1   1   572.0   8.6     172
3  150     8     1   1   1  1106.0  16.6     332
4  150     9     1   1   1  1474.0  20.1     402


In [13]:
# 7. 입력/정규화/트레인테스트 분리
X = df[["역번호", "hour", "평일주말", "상행", "승차", "승하차인원", "혼잡도"]].values
y = df["target"].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [14]:
# 8. 딥러닝 모델 정의&학습
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=40,
    batch_size=64,
    verbose=2
)


Epoch 1/40


c:\Users\user\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29583/29583 - 35s - 1ms/step - loss: 54528.3203 - mae: 172.7578 - val_loss: 52706.8320 - val_mae: 170.1487
Epoch 2/40
29583/29583 - 34s - 1ms/step - loss: 52997.9570 - mae: 170.4597 - val_loss: 52680.3086 - val_mae: 170.1548
Epoch 3/40
29583/29583 - 33s - 1ms/step - loss: 52998.4961 - mae: 170.4568 - val_loss: 52699.9883 - val_mae: 170.1499
Epoch 4/40
29583/29583 - 34s - 1ms/step - loss: 52997.7031 - mae: 170.4532 - val_loss: 52718.4570 - val_mae: 170.1573
Epoch 5/40
29583/29583 - 34s - 1ms/step - loss: 52997.2188 - mae: 170.4545 - val_loss: 52676.9961 - val_mae: 170.1754
Epoch 6/40
29583/29583 - 35s - 1ms/step - loss: 52997.5078 - mae: 170.4551 - val_loss: 52683.5898 - val_mae: 170.2309
Epoch 7/40
29583/29583 - 34s - 1ms/step - loss: 52998.8750 - mae: 170.4565 - val_loss: 52677.3320 - val_mae: 170.1701
Epoch 8/40
29583/29583 - 34s - 1ms/step - loss: 52997.8984 - mae: 170.4560 - val_loss: 52702.9688 - val_mae: 170.1494
Epoch 9/40
29583/29583 - 34s - 1ms/step - loss: 52999.5781 - mae: 1

In [15]:
# 학습 직후 (fit 다음 셀)
model.save("my_dl_model.1line.h5")
import joblib
joblib.dump(scaler, "my_scaler.1line.pkl")


['my_scaler.1line.pkl']

In [16]:
# 9. 예측 함수
def dl_predict(역번호, hour, 평일주말, 상행, 승차, 승하차인원, 혼잡도):
    arr = np.array([[역번호, hour, 평일주말, 상행, 승차, 승하차인원, 혼잡도]])
    arr_scaled = scaler.transform(arr)
    pred = model.predict(arr_scaled)
    return int(pred[0][0])


In [17]:
def user_predict_smart():
    # 입력
    역명 = input("역명: ")
    호선 = input("호선: ")
    방향 = input("방향 (상행/하행): ").strip()
    평일주말_str = input("요일 (평일/주말): ").strip()
    시간 = input("시간 (HH:MM): ").strip()

    # 시간 가공
    hour = int(시간.split(":")[0])
    평일주말 = 1 if 평일주말_str == "주말" else 0
    상행 = 1 if 방향 == "상행" else 0

    # 역번호 자동 조회
    # 최신화 : 승하차/혼잡도 둘 다에 맞추려면 승하차 or 혼잡도 데이터(ex. 승하차_expanded)에서 조회
    # 보통 승하차 데이터에 역명, 호선이 있으므로:
    df_ref = 승하차_expanded
    후보 = df_ref[(df_ref["역명"] == 역명) & (df_ref["호선"].astype(str) == str(호선))]
    if 후보.empty:
        print(f"해당 역명/호선이 데이터에 없습니다.")
        return
    역번호 = int(후보.iloc[0]["역번호"])

    # 승하차 값 탐색
    row_승 = df_ref[
        (df_ref["역번호"] == str(역번호)) &
        (df_ref["방향"] == 방향) &
        (df_ref["평일주말"] == 평일주말_str) &
        (df_ref["구분"] == "승차")
    ]
    row_하 = df_ref[
        (df_ref["역번호"] == str(역번호)) &
        (df_ref["방향"] == 방향) &
        (df_ref["평일주말"] == 평일주말_str) &
        (df_ref["구분"] == "하차")
    ]
    시간컬럼 = 시간 if 시간 in df_ref.columns else f"{hour:02d}:00"
    # 승차/하차 인원 조회(둘 다 있는 경우 합쳐도 되고, 하나만 써도 됨)
    if not row_승.empty and 시간컬럼 in row_승.columns:
        승차인원 = float(row_승.iloc[0][시간컬럼])
    else:
        승차인원 = 0
    if not row_하.empty and 시간컬럼 in row_하.columns:
        하차인원 = float(row_하.iloc[0][시간컬럼])
    else:
        하차인원 = 0
    승하차인원 = 승차인원 + 하차인원 # 혹은 필요에 따라 방법 선택

    # 혼잡도 탐색
    row_혼 = 혼잡도[
        (혼잡도["역번호"] == str(역번호)) &
        (혼잡도["구분"] == 방향) &
        (혼잡도["평일주말"] == 평일주말_str)
    ]
    if not row_혼.empty and 시간컬럼 in row_혼.columns:
        혼잡도_val = float(row_혼.iloc[0][시간컬럼])
    else:
        혼잡도_val = 0

    # 예측
    pred = dl_predict(
        역번호=역번호,
        hour=hour,
        평일주말=평일주말,
        상행=상행,
        승차=1,  # 승차만 예측할지, 승/하차인원합 예측할지는 논리 선택!
        승하차인원=승하차인원,
        혼잡도=혼잡도_val
    )

    # 출력
    print(f"예측: {호선}호선 {역명}({역번호}) {방향} {평일주말_str} {시간}")
    print(f"  승하차 인원(입력값): {승하차인원}, 혼잡도(%): {혼잡도_val:.1f}")
    print(f"  딥러닝 기반 예측 열차 내 인원: {pred} 명")


In [18]:
# 실행 예시
user_predict_smart()

ValueError: invalid literal for int() with base 10: ''

In [ ]:
# 1. 패키지 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import joblib
from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 2. 파일 경로 및 주요 파라미터 별도 선언
승하차_파일 = "../../data/결과/승하차/통합/1호선_승하차인원_통합.csv"
혼잡도_파일 = "../../data/결과/혼잡도/통합/1호선_혼잡도_통합.csv"
정원 = 2000  # 열차 정원

# 3. 데이터 불러오기 및 0패딩 맞춤
def fix_timecols(cols):
    return [c if ':' not in c else c.zfill(5) for c in cols]

def load_data():
    승하차_df = pd.read_csv(승하차_파일, encoding="euc-kr")
    혼잡도_df = pd.read_csv(혼잡도_파일, encoding="euc-kr")
    승하차_df.columns = fix_timecols(승하차_df.columns)
    혼잡도_df.columns = fix_timecols(혼잡도_df.columns)
    return 승하차_df, 혼잡도_df

# 4. 상행/하행 방향 데이터 복제
def duplicate_for_directions(승하차):
    dfs = []
    for direction in ["상행", "하행"]:
        tmp = 승하차.copy()
        tmp['방향'] = direction
        dfs.append(tmp)
    return pd.concat(dfs, ignore_index=True)

# 5. 컬럼 표준화
def standardize_columns(df, do_direction=True):
    df["평일주말"] = df["평일주말"].astype(str).str.strip()
    df["구분"] = df["구분"].astype(str).str.strip()
    df["역번호"] = df["역번호"].astype(str).str.strip()
    if do_direction and "방향" in df.columns:
        df["방향"] = df["방향"].astype(str).str.strip()
    return df

# 6. 벡터 연산 기반 학습 데이터 생성
def build_dl_dataset(승하차, 혼잡도, 정원=2000):
    시간컬럼들 = [col for col in 승하차.columns if ':' in col]
    # key 조합 만들기
    승하차["key"] = (
        승하차["역번호"].astype(str) + "_" +
        승하차["방향"].astype(str) + "_" +
        승하차["평일주말"].astype(str)
    )
    혼잡도["key"] = (
        혼잡도["역번호"].astype(str) + "_" +
        혼잡도["구분"].astype(str) + "_" +
        혼잡도["평일주말"].astype(str)
    )
    # 혼잡도에서 필요한 시간만 추출 후 melt
    혼잡도_long = 혼잡도.melt(id_vars=["key"], value_vars=시간컬럼들, var_name="time", value_name="혼잡도")
    승하차_long = 승하차.melt(id_vars=["key", "역번호", "방향", "평일주말", "구분"],
                            value_vars=시간컬럼들, var_name="time", value_name="승하차인원")
    # merge
    merged = pd.merge(승하차_long, 혼잡도_long, on=["key", "time"], how="inner")
    # feature 생성
    merged["hour"] = merged["time"].str.split(":").str[0].astype(int)
    merged["평일주말_digit"] = merged["평일주말"].map({"평일": 0, "주말": 1})
    merged["상행"] = (merged["방향"] == "상행").astype(int)
    merged["승차"] = (merged["구분"] == "승차").astype(int)
    merged["혼잡도"] = pd.to_numeric(merged["혼잡도"], errors='coerce').fillna(0)
    merged["승하차인원"] = pd.to_numeric(merged["승하차인원"], errors='coerce').fillna(0)
    merged["target"] = (merged["혼잡도"] * 정원 / 100).astype(int)
    cols = ["역번호", "hour", "평일주말_digit", "상행", "승차", "승하차인원", "혼잡도", "target"]
    merged = merged[cols].dropna().reset_index(drop=True)
    merged["역번호"] = merged["역번호"].astype(int)
    return merged

# 7. 입력 값 안전하게 받는 함수
def safe_hour_input():
    while True:
        s = input("시간 (HH:MM): ").strip()
        parts = s.split(":")
        if len(parts) == 2 and parts[0].isdigit():
            return int(parts[0]), f"{int(parts[0]):02d}:00"
        print("시간을 올바른 형식(HH:MM)으로 입력하세요.")

# 8. 사용자 예측 함수 개선
def user_predict_smart(승하차_df, 혼잡도_df, scaler, model):
    역명 = input("역명: ").strip()
    호선 = input("호선: ").strip()
    방향 = input("방향 (상행/하행): ").strip()
    평일주말_str = input("요일 (평일/주말): ").strip()
    hour, 시간컬럼 = safe_hour_input()
    평일주말 = 1 if 평일주말_str == "주말" else 0
    상행 = 1 if 방향 == "상행" else 0

    # 역번호 찾기
    후보 = 승하차_df[(승하차_df["역명"] == 역명) & (승하차_df["호선"].astype(str) == str(호선))]
    if 후보.empty:
        print(f"해당 역명/호선이 데이터에 없습니다.")
        return
    역번호 = int(후보.iloc[0]["역번호"])

    # 승차/하차 인원 찾기
    def get_people(df, 구분):
        row = df[(df["역번호"] == str(역번호)) & (df["방향"] == 방향) & 
                 (df["평일주말"] == 평일주말_str) & (df["구분"] == 구분)]
        if not row.empty and 시간컬럼 in row.columns:
            try:
                return float(row.iloc[0][시간컬럼])
            except Exception:
                return 0
        return 0

    승차인원 = get_people(승하차_df, "승차")
    하차인원 = get_people(승하차_df, "하차")
    승하차인원 = 승차인원 + 하차인원

    # 혼잡도 찾기
    row_혼 = 혼잡도_df[
        (혼잡도_df["역번호"] == str(역번호)) & 
        (혼잡도_df["구분"] == 방향) & 
        (혼잡도_df["평일주말"] == 평일주말_str)
    ]
    혼잡도_val = 0
    if not row_혼.empty and 시간컬럼 in row_혼.columns:
        try:
            혼잡도_val = float(row_혼.iloc[0][시간컬럼])
        except Exception:
            혼잡도_val = 0

    # 예측
    arr = np.array([[역번호, hour, 평일주말, 상행, 1, 승하차인원, 혼잡도_val]])
    arr_scaled = scaler.transform(arr)
    pred = model.predict(arr_scaled, verbose=0)
    print(f"예측: {호선}호선 {역명}({역번호}) {방향} {평일주말_str} {시간컬럼}")
    print(f" 승하차 인원(입력): {승하차인원}, 혼잡도(%): {혼잡도_val:.1f}")
    print(f" 딥러닝 기반 예측 열차 내 인원: {int(pred[0][0])} 명")

# 9. 전체 데이터 전처리 및 학습

print("> 데이터 로딩 및 전처리...")
승하차, 혼잡도 = load_data()
승하차_expanded = duplicate_for_directions(승하차)
승하차_expanded = standardize_columns(승하차_expanded)
혼잡도 = standardize_columns(혼잡도)
df = build_dl_dataset(승하차_expanded, 혼잡도, 정원=정원)
if df.shape[0]==0:
    raise ValueError("학습데이터 생성 실패! 컬럼 또는 값 일치 여부 확인 필요.")
print(f"> 학습데이터 shape: {df.shape}")

# 10. 입력/정규화/트레인테스트 분리
X = df[["역번호", "hour", "평일주말_digit", "상행", "승차", "승하차인원", "혼잡도"]].values
y = df["target"].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# 11. 딥러닝 모델과 학습 (Input 경고 해결 및 EarlyStopping 추가)
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]
model = keras.Sequential([
    keras.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=40,
    batch_size=64,
    verbose=2,
    callbacks=callbacks
)

# 12. 모델/스케일러 저장 (최신 포맷)
model.save("my_dl_model.1line.keras")
joblib.dump(scaler, "my_scaler.1line.pkl")

# 13. 예측 예제 실행
# user_predict_smart(승하차_expanded, 혼잡도, scaler, model)   # ← 실제 실행 시 활성화



> 데이터 로딩 및 전처리...
> 학습데이터 shape: (17560800, 8)
Epoch 1/40
197559/197559 - 160s - 811us/step - loss: 186.3157 - mae: 0.9730 - val_loss: 0.0804 - val_mae: 0.2244
Epoch 2/40
197559/197559 - 158s - 798us/step - loss: 0.3602 - mae: 0.3475 - val_loss: 1.2586 - val_mae: 0.9254
Epoch 3/40
197559/197559 - 161s - 813us/step - loss: 0.2719 - mae: 0.2811 - val_loss: 0.5994 - val_mae: 0.6587
Epoch 4/40
197559/197559 - 158s - 801us/step - loss: 0.2271 - mae: 0.2468 - val_loss: 0.0405 - val_mae: 0.1663
Epoch 5/40
197559/197559 - 162s - 820us/step - loss: 0.1979 - mae: 0.2228 - val_loss: 0.0048 - val_mae: 0.0458
Epoch 6/40
